In [1]:
# all the inmports here
import pandas as pd

count_data_path=(r'C:\Users\manos\Desktop\VU\ML\TCGA_dataset\TCGA_dataset\tcga_rna_count_data_crc.csv') 
prediction_data_path=(r'C:\Users\manos\Desktop\VU\ML\TCGA_dataset\TCGA_dataset\prediction_file_crc.csv')


In [2]:
count_df=pd.read_csv(count_data_path)
pred_df=pd.read_csv(prediction_data_path)

In [3]:
value_counts=pred_df['msi_status'].value_counts()
percentage= value_counts/len(pred_df)*100
print(percentage)

MSS              63.419913
MSI-L            15.800866
MSI-H            14.935065
Indeterminate     0.432900
Name: msi_status, dtype: float64


Convert the MSI-H and MSI_L to MSI

In [4]:
pred_df['msi_status']=pred_df['msi_status'].replace({'MSI-H':0, "MSI-L": 1,'MSS':1})

Categorical to numerical MSI=0 , MSS=1 (BINARY)

In [5]:
value_counts=pred_df['msi_status'].value_counts()
percentage= value_counts/len(pred_df)*100
print(percentage)

1                79.220779
0                14.935065
Indeterminate     0.432900
Name: msi_status, dtype: float64


Remove Indeterminate & unknown Values

In [6]:
pred_df = pred_df[pred_df['msi_status'] != 'Indeterminate']
pred_df = pred_df[pred_df['msi_status'].isin([0,1])]

Get dumies to convert categorical to numerical

In [7]:
pred_df_num = pd.get_dummies(pred_df, columns=['TP53'])
pred_df_num = pd.get_dummies(pred_df_num, columns=['KRAS'])
pred_df_num = pd.get_dummies(pred_df_num, columns=['BRAF'])
pred_df_num = pd.get_dummies(pred_df_num, columns=['APC'])
pred_df_num = pd.get_dummies(pred_df_num, columns=['TTN'])

Enumarate Sample ids  in a dictionary save the dict in a file replace ids with keys

In [8]:
id_to_num = {}
for i, id in enumerate(pred_df_num['Unnamed: 0'].unique()):
    id_to_num[id] = i

# Save the dictionary to a file
import json
with open('id_to_num.json', 'w') as f:
    json.dump(id_to_num, f)

# Replace the IDs with the corresponding numbers in the DataFrame
pred_df_num['Unnamed: 0'] = pred_df_num['Unnamed: 0'].map(id_to_num)

In [9]:
pred_df_num

,Unnamed: 0,TBL,msi_status,fraction_genome_altered,aneuploidy_score,TMB,TP53_SNV,TP53_WT,KRAS_SNV,KRAS_WT,BRAF_SNV,BRAF_WT,APC_SNV,APC_WT,TTN_SNV,TTN_WT
0,0,19,1,0.114119,6,118,0,1,0,1,1,0,1,0,0,1
1,1,38,1,0.311631,12,81,1,0,1,0,0,1,0,1,0,1
3,2,87,1,0.422921,13,159,1,0,1,0,0,1,1,0,1,0
4,3,28,1,0.357734,12,72,1,0,0,1,0,1,1,0,0,1
5,4,3,0,0.049432,2,1828,0,1,1,0,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,430,104,1,0.309643,9,209,1,0,1,0,0,1,1,0,1,0
458,431,105,1,0.270808,10,115,1,0,0,1,0,1,1,0,0,1
459,432,82,1,0.408573,15,143,1,0,1,0,0,1,1,0,0,1
460,433,115,1,0.178141,6,150,1,0,0,1,0,1,0,1,0,1


Balancing (Requires further exploration)

In [10]:
# from imblearn.over_sampling import SMOTE
# from imblearn.under_sampling import RandomUnderSampler

# # Separate majority and minority classes
# majority_class = pred_df_num[pred_df_num['msi_status'] == 1]
# minority_class = pred_df_num[pred_df_num['msi_status'] == 0]

# # Remove unknown and intermediate values from minority class
# minority_class = minority_class[minority_class['msi_status'].isin([0,1])]

# # Apply SMOTE oversampling to minority class
# oversampler = SMOTE(random_state=42)
# minority_oversampled, minority_labels_oversampled = oversampler.fit_resample(minority_class.drop('msi_status', axis=1), minority_class['msi_status'])

# # Apply RandomUnderSampler to majority class
# undersampler = RandomUnderSampler(random_state=42)
# majority_undersampled, majority_labels_undersampled = undersampler.fit_resample(majority_class.drop('msi_status', axis=1), majority_class['msi_status'])

# # Combine oversampled and undersampled data
# df_balanced = pd.concat([majority_undersampled, minority_oversampled])
# df_balanced['msi_status'] = pd.concat([majority_labels_undersampled, minority_labels_oversampled])

# # Verify the class distribution of the balanced data
# print(df_balanced['msi_status'].value_counts())


# **Prepro Count Data**

Remove Genes with Zero counts in all Samples

In [11]:
# Check which genes have zero counts in all samples
zero_count_genes = count_df.columns[count_df.eq(0).all(axis=0)]

# Remove genes with zero counts from dataframe
count_df = count_df.drop(columns=zero_count_genes)

In [12]:
count_df

,Unnamed: 0,TCGA-CK-5912-01A,TCGA-QG-A5Z2-01A,TCGA-AG-3898-01A,TCGA-G4-6299-01A,TCGA-AG-4008-01A,TCGA-NH-A8F8-01A,TCGA-AD-6548-01A,TCGA-AA-A02Y-01A,TCGA-EI-6514-01A,...,TCGA-CL-5918-01A,TCGA-AG-A01Y-01A,TCGA-AG-A014-01A,TCGA-AG-A016-01A,TCGA-AA-3846-01A,TCGA-CA-5797-01A,TCGA-AA-3860-01A,TCGA-CK-4951-01A,TCGA-EI-6507-01A,TCGA-AA-3858-01A
0,ND4,282779,527763,351929,287719,190175,534458,258680,609116,1788482,...,967151,253949,357582,193103,378175,241368,174340,344243,187689,120685
1,COX1,174425,550980,221923,77856,126014,573269,334800,988344,882782,...,503826,178148,277609,169953,328256,270218,159090,244393,186843,113810
2,COX2,215346,455961,219310,224206,121250,539229,239678,729632,770006,...,688044,100534,233873,148345,196468,253742,107697,196681,104451,99580
3,COX3,215935,224912,194021,249685,163588,640377,215029,496405,509296,...,656321,119946,133931,76543,247324,265639,130445,200891,90755,137800
4,ACTB,114483,273373,111800,266851,94666,199718,508419,342245,275784,...,264552,123410,150556,171336,90398,371604,124204,210810,235117,150755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33374,PRAMEF13,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33375,OR8B4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33376,OR1S1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33377,PRAMEF26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


 Remove POLE POLD genes from count data

In [13]:
count_df_01 = count_df[count_df.iloc[:,0] != 'POLE']
count_df_02 = count_df_01[count_df_01.iloc[:,0] != 'POLD1']

In [14]:
count_df_norm=count_df_02.iloc[:,1:]

One common normalization technique for RNA-seq data is called "library size normalization". This involves dividing the raw counts of each sample by the total number of reads in that sample, and then multiplying the result by a scaling factor (e.g., 1 million) to get "counts per million" (CPM) or "reads per million" (RPM).

In [15]:
# Calculate library sizes (total number of reads) for each sample
library_sizes = count_df_norm.sum(axis=0)

# Normalize counts by library size and scale to CPM
cpm_df = count_df_norm.divide(library_sizes, axis=1) * 1e6

add back the names of the genes

In [16]:
# Get the names of the genes
gene_names = count_df_02.iloc[:, 0]

# Insert the gene names at the beginning of the cpm_df dataframe
cpm_df.insert(loc=0, column='Gene Name', value=gene_names)

In [19]:
print(cpm_df.shape)
print(pred_df_num.shape)

(33377, 463)
(435, 16)


Transpose counts df

In [20]:
cpm_df = cpm_df.transpose()

# Now remove the samples from the count_df that we dont have msi values for

Add sample names back to pred_df

In [21]:
# Load the mapping dictionary from the file
import json
with open('id_to_num.json', 'r') as f:
    num_to_id = json.load(f)

# Reverse the mapping dictionary
num_to_id = {v: k for k, v in num_to_id.items()}

# Map the numbers back to the gene names
pred_df_num['Unnamed: 0'] = pred_df_num['Unnamed: 0'].map(num_to_id)

Remove the samples

In [22]:
# Set the first row as column names
cpm_df = cpm_df.set_axis(cpm_df.iloc[0], axis=1)

# Dropa the first row after setting column names
cpm_df = cpm_df[1:]

In [23]:
pred_df_num = pred_df_num.set_index('Unnamed: 0')

In [24]:
# Get the common sample names between the two dataframes
common_samples = set(cpm_df.index).intersection(set(pred_df_num.index))

print("Number of common samples:", len(common_samples))
print("Samples in cpm_df but not in pred_df_num:", set(cpm_df.index) - common_samples)
print("Samples in pred_df_num but not in cpm_df:", set(pred_df_num.index) - common_samples)

# Reduce cpm_df to only the common samples
cpm_df_common = cpm_df.loc[common_samples]

# Reduce pred_df_num to only the common samples
pred_df_common = pred_df_num.loc[common_samples]


Number of common samples: 435
Samples in cpm_df but not in pred_df_num: {'TCGA-AG-4015-01A', 'TCGA-AA-A01R-01A', 'TCGA-A6-6650-01A', 'TCGA-A6-2684-01A', 'TCGA-NH-A8F7-01A', 'TCGA-AA-A02Y-01A', 'TCGA-A6-3809-01A', 'TCGA-A6-6780-01A', 'TCGA-A6-5656-01A', 'TCGA-AA-A03F-01A', 'TCGA-A6-2681-01A', 'TCGA-AA-3666-01A', 'TCGA-4T-AA8H-01A', 'TCGA-AG-3896-01A', 'TCGA-D5-6540-01A', 'TCGA-F5-6814-01A', 'TCGA-A6-5659-01A', 'TCGA-A6-2677-01A', 'TCGA-AG-3885-01A', 'TCGA-AA-A01S-01A', 'TCGA-A6-2680-01A', 'TCGA-AA-A02O-01A', 'TCGA-AA-A01T-01A', 'TCGA-AA-3956-01A', 'TCGA-A6-3810-01A', 'TCGA-AG-A032-01A', 'TCGA-CA-6718-01A'}
Samples in pred_df_num but not in cpm_df: set()


C:\Users\manos\AppData\Local\Temp\ipykernel_10628\3429427196.py:9: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  cpm_df_common = cpm_df.loc[common_samples]
C:\Users\manos\AppData\Local\Temp\ipykernel_10628\3429427196.py:12: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  pred_df_common = pred_df_num.loc[common_samples]


In [25]:
cpm_df_common

Gene Name,ND4,COX1,COX2,COX3,ACTB,EEF1A1,CYTB,ACTG1,ND2,ATP6,...,MIR8060,RNU6-156P,MIR6082,RN7SL82P,TP53TG3F,PRAMEF13,OR8B4,OR1S1,PRAMEF26,DUX4L25
TCGA-AZ-6601-01A,6968.616215,9949.728089,3781.654651,3638.434937,7595.645204,4978.12472,2316.746585,5493.686519,1728.7832,2118.043296,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-F4-6856-01A,1774.611905,3275.289249,1597.840212,922.308507,4797.214374,3257.700025,517.61566,4782.544961,817.511915,534.430957,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-AA-3672-01A,4940.128371,6707.386835,3139.035986,2852.450146,15874.374401,12388.35276,2729.144301,10674.852638,993.088231,3099.437768,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-CA-5256-01A,2925.163468,3038.052058,2373.249773,2593.514081,3452.603352,8482.431123,1515.914597,3014.789408,1109.214971,1600.27825,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-CM-6674-01A,6421.742017,6775.857766,6085.891076,6089.677997,7490.890265,5006.644372,2595.74109,4097.706064,2181.67864,3352.687333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-D5-6535-01A,11315.600706,11395.623394,8609.401985,4750.968933,5227.132795,3678.988236,5492.812739,4622.147416,5519.39459,4709.283232,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-A6-4105-01A,7988.938089,7291.57955,3949.049829,4493.865662,5157.914391,5108.403032,2241.516408,4888.691904,3582.403305,2856.459413,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-AA-3814-01A,9420.190396,9738.402014,6482.81003,6066.856541,6394.173229,7404.200791,7678.962168,4509.254288,2740.075611,2691.755215,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-CM-5861-01A,9421.140802,14935.75064,7302.59925,6036.499856,4940.720754,6909.741642,3339.959114,4506.20162,2600.576,4166.74425,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Splitng the data into Train, Test, Validation Sets

In [26]:
from sklearn.model_selection import train_test_split
pred_df_common['msi_status']=pred_df_common['msi_status'].astype(int)
X = cpm_df_common.iloc[:,1:]
y = pred_df_common['msi_status']

##  selected random state 42 so that now you get the same result each time you run the function
X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.79, random_state=42)

## validation data is the same size as your test set, but you can make it a littler smaller to get some more training data
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5, random_state=42)


In [27]:
print('Xtrain',X_train.shape)
print('ytrain',y_train.shape)
print('Xtest',X_valid.shape)
print('Xvalid',X_test.shape)



Xtrain (343, 33376)
ytrain (343,)
Xtest (46, 33376)
Xvalid (46, 33376)


In [28]:
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from matplotlib import pyplot


In [29]:
X_train_val=X_train.astype(int)
y_train_val=y_train.astype(int)
# Calculate the correlation matrix between the X input features and y output feature
corr_matrix = X_train_val.corrwith(y_train_val)
#drop nan values
corr_matrix_nan=corr_matrix.dropna()
#convert to absolute
corr_matrix_abs=corr_matrix_nan.abs()
#sort values
coor_matrix_sorted=corr_matrix_abs.sort_values(ascending=False)
#set threshold
threshold = 0.0
#select features
Selected_features=coor_matrix_sorted[coor_matrix_sorted>=threshold]
Selected_features


Gene Name
TRIM7      6.880843e-01
MLH1       6.213414e-01
GNLY       6.038499e-01
FECH       5.795658e-01
HPSE       5.791100e-01
               ...     
DCUN1D5    9.091038e-05
RCE1       6.424163e-05
UGT2B15    5.895388e-05
TRPC4      1.893514e-16
ZNF593     2.268327e-17
Length: 23306, dtype: float64

In [99]:
#keep only the selected features in Xtrain, xtest,xvalid
X_train  =X_train.loc[:, X_train.columns.isin(Selected_features.index)]
X_test = X_test.loc[:, X_test.columns.isin(Selected_features.index)]
X_valid = X_valid.loc[:, X_valid.columns.isin(Selected_features.index)]

In [30]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
# Define X and y


# Define the estimator for RFE
estimator=DecisionTreeClassifier()

# Define the number of features to select
n_features = 23306

# Perform RFE
rfe = RFE(estimator, n_features_to_select=n_features)
rfe.fit(X_train, y_train)

# Get the selected features
selected_features = X_train.columns[rfe.support_]
# Print the progress
for i, feature in enumerate(selected_features):
    print(f"{i+1}/{len(selected_features)} selected features: {feature}")

print('Selected Features:')

In [ ]:
#select features in the datasets
X_train  =X_train.loc[:, X_train.columns.isin(selected_features)]
X_test = X_test.loc[:, X_test.columns.isin(selected_features)]
X_valid = X_valid.loc[:, X_valid.columns.isin(selected_features)]

In [102]:
# with open('selected_features from 33000 to 1000.txt', 'w') as f:
#     # Write the selected features to the file, separated by commas
#     f.write(','.join(selected_features))

In [ ]:
#export dfs for future use
X_train.to_csv('X_train1.csv', index=False)
X_test.to_csv('X_test1.csv', index=False)
X_valid.to_csv('X_valid1.csv', index=False)
y_train.to_csv('y_train1.csv', index=False)
y_test.to_csv('y_test1.csv', index=False)
y_valid.to_csv('y_valid1.csv', index=False)
